In [9]:
import os
import numpy as np
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

In [11]:
GOOGLE_CLOUD_REGION='australia-southeast1'
ENDPOINT_NAME='prediction-penguin-vertex-pipelines'
GOOGLE_CLOUD_PROJECT='som-k8s'

In [12]:
os.environ['REGION']=GOOGLE_CLOUD_REGION
os.environ['ENDPOINT_NAME']=ENDPOINT_NAME

create a dummy datapoint

In [ ]:
%%writefile example_input.json
{"instances": [
  {"culmen_length_mm":[0.71],"culmen_depth_mm":[0.38],"flipper_length_mm":[0.98],"body_mass_g":[0.78]},
  {"culmen_length_mm":[0.22],"culmen_depth_mm":[0.18],"flipper_length_mm":[0.18],"body_mass_g":[0.38]}
]}


make predicitons using glcoud ai utility

In [6]:
%%bash
echo $REGION
ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)' --filter=display_name=${ENDPOINT_NAME})
echo $ENDPOINT_ID
gcloud ai endpoints predict $ENDPOINT_ID --region=$REGION --json-request=example_input.json

australia-southeast1
8081713929362866176
[[-1.22456849, -2.00882196, 6.0371232], [0.718155444, -0.00462970138, 0.345121533]]


Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Using endpoint [https://australia-southeast1-prediction-aiplatform.googleapis.com/]


Make predicitons using curl request

In [7]:
%%bash
PROJECT=$(gcloud config get-value project)
ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)' --filter=display_name=${ENDPOINT_NAME})

curl -X POST \
  -H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @example_input.json \
  "https://${REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT}/locations/${REGION}/endpoints/${ENDPOINT_ID}:predict"

{
  "predictions": [
    [
      -1.22456849,
      -2.00882196,
      6.0371232
    ],
    [
      0.718155444,
      -0.00462970138,
      0.345121533
    ]
  ],
  "deployedModelId": "2151775041883209728",
  "model": "projects/41931926481/locations/australia-southeast1/models/2261959301125898240",
  "modelDisplayName": "v1718844611",
  "modelVersionId": "1"
}


Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   586    0   364  100   222   2510   1531 --:--:-- --:--:-- --:--:--  4041


make prediciton with a python client

In [13]:
# prgramattically find the endpoint id to use it for prediciton
import subprocess

result = subprocess.run(
    [
        "bash", "-c",
        f"ENDPOINT_ID=$(gcloud ai endpoints list --region={GOOGLE_CLOUD_REGION} --format='value(ENDPOINT_ID)' --filter=display_name={ENDPOINT_NAME}); echo $ENDPOINT_ID"
    ],
    capture_output=True, text=True
)

ENDPOINT_ID = result.stdout.strip()
print(f"The value of ENDPOINT_ID is: {ENDPOINT_ID}")

# python client
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)
print(response.predictions[0])
# Uses argmax to find the index of the maximum value.

The value of ENDPOINT_ID is: 8081713929362866176
[-1.22456837, -2.00882196, 6.03712368]
